HW4 Tardigrades.

1. Obtaining data \
1.1 Genomic: `wget -o data/Tardi_assebled_genome.fna.gz http://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/949/185/GCA_001949185.1_Rvar_4.0/GCA_001949185.1_Rvar_4.0_genomic.fna.gz` \
1.2 AUGUSTUS results: https://drive.google.com/file/d/1hCEywBlqNzTrIpQsZTVuZk1S9qKzqQAq/view?usp=sharing \
1.3 GFF: https://drive.google.com/file/d/12ShwrgLkvJIYQV2p1UlXklmxSOOxyxj4/view?usp=sharing \
1.4 getAnnoFasta.pl script: http://augustus.gobics.de/binaries/scripts/getAnnoFasta.pl \
1.5 Tandem Mass Spectrometry results: https://disk.yandex.ru/d/xJqQMGX77Xueqg \

2. Get number of obtained proteins \
`grep ">" data/augustus.whole.aa | wc -l`
-> 16435 \
2.1 To narrow down the amount of genes we can look for those which take their place in a nucleus, thus having a NLS (Nuclear Localization Signals). Classical NLS (cNLS): This type of signal is recognized by importin α and importin β proteins. The classical NLS is usually a short stretch of basic amino acids, such as lysine or arginine. The consensus sequence for classical NLS is typically (K/R)4-5(K/R)X(K/R).

3. Do a local alignment-based search \
3.1 *Option a* \
`conda install -c bioconda blast` \
`makeblastdb -in data/peptides.fa -dbtype prot  -out data/blastp_tardigrade_protein_db` \
`blastp -db data/blastp_tardigrade_protein_db -query data/augustus.whole.aa -outfmt 6 -out data/blastp_tardigrade_filtered_proteins` \
Column names format 6: qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore \
3.2 *Option b* \
`conda install -c bioconda diamond` \
`diamond makedb --in data/peptides.fa  --db data/tardigrade_protein_db` \
`diamond blastp -d data/tardigrade_protein_db.dmnd -q data/augustus.whole.aa  -f 6 -o data/diamond_tardigrade_filtered_proteins --very-sensitive` \
Used option b. From TMS created a local database with peptides. Then blasted against fasta file with all proteins.

4. Localization prediction \
Get fasta's for filtered genes \
for 3a:
- filter only unique gene names, get top 5000 of them
`awk '!seen[$1]++' data/blastp_tardigrade_filtered_proteins | head -n 5000 data/blastp_tardigrade_filtered_proteins_unique | awk '{print $1}' > data/blastp_tardigrade_filtered_proteins_unique_5000`
- get fasta sequences from the original fasta for every unique gene name
`awk 'NR==FNR{genes[$1]; next} /^>/{if (substr($1, 2) in genes) print; p=0} p{print} /^>/ && (substr($1, 2) in genes){p=1}' data/blastp_tardigrade_filtered_proteins_unique_5000 data/augustus.whole.aa > data/blastp_augustus.whole_filtered_proteins.aa` \

for 3b:
- `awk 'NR==FNR{genes[$1]; next} /^>/{if (substr($1, 2) in genes) print; p=0} p{print} /^>/ && (substr($1, 2) in genes){p=1}' data/diamond_tardigrade_filtered_proteins data/augustus.whole.aa > data/diamond_augustus.whole_filtered_proteins.aa` \
4a. WoLF PSORT  (https://wolfpsort.hgc.jp/) predicts the subcellular localization of proteins. PSORT predicts cellular localization of proteins based on the presence of a signal peptide on their N-terminus. \
Results: \
g4106.t1 details E.R.: 14.5, E.R._golg: 9.5, extr: 7, golg: 3.5, lyso: 3, pero: 2, plas: 1, mito: 1 \
g12510.t1 details plas: 29, cyto: 3 \
(see pdfs) \
4b. TargetP (https://services.healthtech.dtu.dk/service.php?TargetP-2.0) also predicts the subcellular localization of eukaryotic proteins. The location assignment is based on the predicted presence of any of the N-terminal presequences: chloroplast transit peptide (cTP), mitochondrial targeting peptide (mTP) or secretory pathway signal peptide (SP). \
Results: \
(see zip and pdf)

5. BLAST
BLAST (https://blast.ncbi.nlm.nih.gov/Blast.cgi) protein sequences against the “UniProtKB/Swiss-Prot” database.
